In [ ]:
!which python

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
# basic packages
import os
import re
import sys
import datetime
from typing import List, Dict, Tuple, Optional, Any
from itertools import combinations
from pathlib import Path
import glob
#import yaml
import tqdm
import multiprocessing as mp
from itertools import combinations, product

In [ ]:
# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# bioinformatics
import pandas as pd
from Bio.Seq import MutableSeq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

In [ ]:
ROOT_dir = Path(os.path.abspath(os.path.join(Path("../")))).__str__()
if ROOT_dir not in sys.path:
    sys.path.append(ROOT_dir)

In [ ]:
SCRIPTS_dir = ROOT_dir+ "/scripts/"
if SCRIPTS_dir not in sys.path:
    sys.path.append(SCRIPTS_dir)

In [ ]:
print(sys.path)

In [ ]:
from serialize_mapping_ import read_mapping, wrapper_read_site_mapping

In [ ]:
mapping: str = "(((Procavia_T:0.120422:T,Loxodonta_T:0.0603659:T)_T:0.0425276:T,Echinops_T:0.226539:T)_T:0.0319915:T,(Dasypus_T:0.101122:T,Choloepus_T:0.0851804:T)_T:0.0498077:T,(((Equus_T:0.0779582:T,(Vicugna_C:0.0907068:C:0.00917933:T,(Sus_T:0.119916:T,(Tursiops_T:0.0426887:T,(Ovis_T:0.0145964:T,Bos_T:0.0304041:T)_T:0.0727613:T)_T:0.00625514:T)_T:0.0122151:T)_T:0.0250573:T)_T:0.00566655:T,((Pteropus_T:0.106978:T,Myotis_T:0.118461:T)_T:0.0114075:T,((Sorex_T:0.198182:T,Erinaceus_T:0.174272:T)_T:0.0430721:T,(Felis_T:0.0839751:T,(Canis_T:0.0699605:T,(Ailuropoda_T:0.052623:T,Mustela_T:0.103226:T)_T:0.0185596:T)_T:0.0103581:T)_T:0.0279674:T)_T:0.00335302:T)_T:0.00079142:T)_T:0.0110501:T,(((Otolemur_T:0.158654:T,Microcebus_C:0.145337:C:0.0178805:T)_T:0.0504843:T,(Tarsius_T:0.330875:T,(((Nomascus_T:0.0264005:T,(Pongo_T:0.013838:T,((Pan_T:0.000132753:T,Homo_T:0.00809315:T)_T:0.00416328:T,Gorilla_T:0.0077017:T)_T:0.00629409:T)_T:0.000696043:T)_T:0.00146144:T,((Papio_T:0.0182087:T,Macaca_T:0.0019362:T)_T:0.00258712:T,Chlorocebu_T:0.00882445:T)_T:0.0191547:T)_T:0.0248701:T,Callithrix_T:0.0475918:T)_T:0.0516841:T)_T:0.00522412:T)_T:0.0187992:T,(Tupaia_T:0.210967:T,((Oryctolagu_T:0.2021:T,Ochotona_T:0.209093:T)_T:0.165137:T,(Ictidomys_A:0.0627069:A:0.0563752:T,(Cavia_T:0.189718:T,((Rattus_T:0.0571824:T,Mus_T:0.0839931:T)_T:0.193646:T,Dipodomys_T:0.149537:T)_T:0.0346516:T)_T:0.00956086:T)_T:0.0328611:T)_T:0.0102136:T)_T:0.0081004:T)_T:0.0277304:T)_T:0.022273:T)_T;"

In [ ]:
read_mapping(mapping=mapping, metadata={})

In [ ]:
mapping_a: pd.DataFrame = wrapper_read_site_mapping(input=ROOT_dir + "/data/mappings/WDR91-GTRG4-A_0.map" , metadata={"mapping_type":0})
mapping_b: pd.DataFrame = wrapper_read_site_mapping(input=ROOT_dir + "/data/mappings/WDR91-GTRG4-A_1.map" , metadata={"mapping_type":0})

In [ ]:
mapping_a

In [ ]:
from scripts.generate_two_sites_mapping_ import generate_two_sites_mapping

In [ ]:
input_dir=ROOT_dir+"/data/mappings/"
pattern='WDR91-GTRG4-A_*.pickle'
generate_two_sites_mapping(input_dir=input_dir,\
                           pattern=pattern,list_of_paired_sites="|".join(["0-1","1-2","2-3"]),\
                           output_dir="/tmp/")

In [ ]:
from generate_two_sites_mapping_ import generate_map_to_list_of_tuples

In [ ]:
mapping_a["list_of_tuples"] = mapping_a["mapping"].apply(
            lambda x: generate_map_to_list_of_tuples(x)
        )
mapping_b["list_of_tuples"] = mapping_b["mapping"].apply(
            lambda x: generate_map_to_list_of_tuples(x)
        )

In [ ]:
mapping_a

In [ ]:
for i in mapping_a.iterrows():
    if i[1].list_of_tuples is not None:
        if len(i[1].list_of_tuples) > 2: 
            print(i)
            break

In [ ]:
mapping_a.dropna(axis=0, inplace=True)
mapping_b.dropna(axis=0, inplace=True)

In [ ]:
mapping_a["list_of_tuples_"] = mapping_a["list_of_tuples"].map(
            lambda x: [(i, j, 0) for i, j in x]
        )
mapping_b["list_of_tuples_"] = mapping_b["list_of_tuples"].map(
    lambda x: [(i, j, 1) for i, j in x]
)

In [ ]:
joint_mapping: pd.DataFrame = pd.DataFrame(
                data=mapping_a["list_of_tuples_"] + mapping_b["list_of_tuples_"]
            )

In [ ]:
joint_mapping

In [ ]:
# sorting on dwell time
joint_mapping["list_of_tuples_"] = joint_mapping["list_of_tuples_"].map(
    lambda x: sorted(x, key=lambda tup: float(tup[1]))
)

In [ ]:
joint_mapping

In [ ]:
assert (
    mapping_a.columns.to_list() == mapping_b.columns.to_list()
), "someting wrong with raw mapping columns"

In [ ]:
c: List[str] = mapping_a.columns.to_list()
for i in c:
    if i not in ["list_of_tuples_", "mapping", "list_of_tuples"]:
        joint_mapping[i] = mapping_a[i]

In [ ]:
joint_mapping

In [ ]:
from generate_two_sites_mapping_stats_ import two_states_mapping_generator

In [ ]:
joint_mapping["joint_mapping"] = joint_mapping["list_of_tuples_"].apply(
            lambda x: list(two_states_mapping_generator(x))
        )

In [ ]:
list(joint_mapping.iloc[0,[0,6]])

In [ ]:
list_of_chainID = joint_mapping.chainID.to_list()

In [ ]:
from typing import Iterable
# accepting only single point mutations
nuc: List[str] = ["A", "C", "G", "T"]
dinuc: Iterable[Tuple[str, str]] = list(product(nuc, nuc))
dinuc_sub: List[str] = [
    "".join(i[0]) + ">" + "".join(i[1])
    for i in product(dinuc, dinuc)
    if "".join(i[0]) != "".join(i[1])
    and (
        (
            "".join(i[0])[0] == "".join(i[1])[0]
            and "".join(i[0])[1] != "".join(i[1])[1]
        )
        or (
            "".join(i[0])[1] == "".join(i[1])[1]
            and "".join(i[0])[0] != "".join(i[1])[0]
        )
    )
]

In [ ]:
assert len (dinuc) == 16

In [ ]:
dict_of_N: Dict[int, Dict[str, int]] = {}
dict_of_T: Dict[int, Dict[str, float]] = {}

for chainID in list_of_chainID:
    dict_of_N_: Dict[str, int] = {i: 0 for i in dinuc_sub}
    dict_of_T_: Dict[str, float] = {"".join(i): 0.0 for i in dinuc}
    
    # print(joint_mapping.loc[
    #     joint_mapping["chainID"] == chainID]["list_of_tuples_"])
    
    for row in joint_mapping.loc[
        joint_mapping["chainID"] == chainID
    ].iterrows():
        if len(row[1].joint_mapping) > 0:
            dwell_time_, state_from = row[1].joint_mapping[0]
            dict_of_T_[state_from] += dwell_time_
            # print(dwell_time_, state_from)
            for dwell_time, state_to in row[1].joint_mapping[1:]:
                dict_of_N_[state_from + ">" + state_to] += 1
                dict_of_T_[state_from] += dwell_time
                print(row[1].joint_mapping[0],row[1].joint_mapping[1])
                print(dwell_time, state_from, state_to)
                state_from = state_to
                # print(row)
                
                

    dict_of_N[chainID] = dict_of_N_
    dict_of_T[chainID] = dict_of_T_

In [ ]:
joint_mapping.loc[joint_mapping["chainID"] == chainID].iloc[14]

In [ ]:
for row in joint_mapping.loc[joint_mapping["chainID"] == chainID].iterrows():
    #print(row[1].joint_mapping)
    for dwell_time, state_to in row[1].joint_mapping[1:]:
        print(dwell_time,state_to)

In [ ]:
pd.DataFrame.from_dict(data=dict_of_N).mean(axis=1)

In [ ]:
pd.DataFrame.from_dict(data=dict_of_T).mean(axis=1)

In [ ]:
list_of_tuples_ = [('A', 0.0, 0), ('T', 0.0, 1), ('A', 0.173679, 0), ('T', 0.173679, 1)]
list(two_states_mapping_generator(list_of_tuples_))

In [ ]:
list_of_tuples_ = [('A', 0.0, 0), ('T', 0.0, 1), ('G', 0.173679, 0), ('T', 0.173679, 1),('C', 0.193679, 1)]
list(two_states_mapping_generator(list_of_tuples_))